In [42]:
import tensorflow as tf
from tensorflow.keras import models,layers
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications import InceptionResNetV2

In [43]:
data_dir = '/kaggle/input/disasterdetection/data_disaster_types/train'
val_data_dir = '/kaggle/input/disasterdetection/data_disaster_types/dev'
test_dir = '/kaggle/input/disasterdetection/data_disaster_types/test'

In [44]:
import os



# Get the list of class folders
class_folders = [folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))]

# Iterate through each class folder and count the number of images
for class_folder in class_folders:
    class_path = os.path.join(data_dir, class_folder)
    num_images = len(os.listdir(class_path))
    print(f"Class: {class_folder}, Number of Images: {num_images}")

Class: flood, Number of Images: 2336
Class: hurricane, Number of Images: 1444
Class: other_disaster, Number of Images: 1132
Class: fire, Number of Images: 1270
Class: landslide, Number of Images: 940
Class: earthquake, Number of Images: 2058
Class: not_disaster, Number of Images: 3666


In [45]:
Imagesize = 224


seed = 123

training_dataset= tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=(Imagesize,Imagesize),
    batch_size=32,
    shuffle=True,
    seed=seed )

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    val_data_dir,
    image_size=(Imagesize,Imagesize),
    batch_size=32,
    shuffle=True,
    seed=seed )

test_dataset  = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(Imagesize,Imagesize),
    batch_size=32,
    shuffle=True,
    seed=seed )

Found 12846 files belonging to 7 classes.
Found 1470 files belonging to 7 classes.
Found 3195 files belonging to 7 classes.


In [46]:
rescale = tf.keras.Sequential([
    layers.Rescaling(1./255),
])
data_augmentation = tf.keras.Sequential([
    layers.RandomContrast(0.1),  
    layers.RandomFlip(mode='horizontal_and_vertical'),  
    layers.RandomRotation(factor=0.2),  
    
])

In [47]:

base_model = InceptionResNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model2 = InceptionV3(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model3 = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

In [48]:
def create_model(base_model):
    
    model = models.Sequential()

   
    for layer in base_model.layers[:-5]:
        layer.trainable = False

    model = models.Sequential()

   
    model.add(rescale)
    model.add(data_augmentation)

    model.add(base_model)

    
    model.add(layers.GlobalAveragePooling2D())
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dense(1024, activation='relu'))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dense(7, activation='softmax'))  
    input  =  (32,224,224,3)
    model.build(input_shape = input)
    
    optimizer = Adam(learning_rate=0.0001)

    model.compile(
    optimizer=optimizer,
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
    )
    
    print(model.summary())

    return model

## InceptionResNetV2


In [49]:
model = create_model(base_model)

Model: "sequential_14"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_11 (Sequential)      │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_12 (Sequential)      │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_resnet_v2             │ (32, 5, 5, 1536)       │    54,336,736 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_5      │ (32, 1536)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (32, 1536)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (32, 1024)             │     1,573,888 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (32, 1024)             │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_27 (Dense)                │ (32, 512)              │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_7 (Dropout)             │ (32, 512)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_28 (Dense)                │ (32, 128)              │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_29 (Dense)                │ (32, 7)                │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 57,551,591 (219.54 MB)

 Trainable params: 7,345,191 (28.02 MB)

 Non-trainable params: 50,206,400 (191.52 MB)

None


In [50]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor='val_accuracy',   
    min_delta=0.001,          
    patience=10,              
    verbose=1,               
    mode='max',             
    restore_best_weights=True
)

In [51]:
record = model.fit(
    training_dataset,
    batch_size=32,
    epochs=40,
    verbose=1,
    validation_data=validation_dataset,
    shuffle=True,
    callbacks=[early_stopping]      
)

Epoch 1/40
402/402 ━━━━━━━━━━━━━━━━━━━━ 134s 262ms/step - accuracy: 0.4104 - loss: 1.5827 - val_accuracy: 0.6476 - val_loss: 1.0270
Epoch 2/40
402/402 ━━━━━━━━━━━━━━━━━━━━ 92s 228ms/step - accuracy: 0.6017 - loss: 1.1359 - val_accuracy: 0.6714 - val_loss: 0.9711
Epoch 3/40
402/402 ━━━━━━━━━━━━━━━━━━━━ 91s 227ms/step - accuracy: 0.6288 - loss: 1.0483 - val_accuracy: 0.6633 - val_loss: 0.9442
Epoch 4/40
402/402 ━━━━━━━━━━━━━━━━━━━━ 91s 227ms/step - accuracy: 0.6604 - loss: 0.9832 - val_accuracy: 0.6653 - val_loss: 0.9211
Epoch 5/40
402/402 ━━━━━━━━━━━━━━━━━━━━ 91s 227ms/step - accuracy: 0.6855 - loss: 0.9064 - val_accuracy: 0.6816 - val_loss: 0.8885
Epoch 7/40
402/402 ━━━━━━━━━━━━━━━━━━━━ 92s 228ms/step - accuracy: 0.6940 - loss: 0.8838 - val_accuracy: 0.6884 - val_loss: 0.8863
Epoch 8/40
402/402 ━━━━━━━━━━━━━━━━━━━━ 91s 227ms/step - accuracy: 0.7018 - loss: 0.8738 - val_accuracy: 0.6857 - val_loss: 0.8942
Epoch 9/40
402/402 ━━━━━━━━━━━━━━━━━━━━ 142s 228ms/step - accuracy: 0.7089 - loss:

## InceptionV3

In [62]:
model2 = create_model(base_model2)


Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_11 (Sequential)      │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_12 (Sequential)      │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ inception_v3 (Functional)       │ (32, 5, 5, 2048)       │    21,802,784 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_9      │ (32, 2048)             │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_14 (Dropout)            │ (32, 2048)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_45 (Dense)                │ (32, 1024)             │     2,098,176 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_46 (Dense)                │ (32, 1024)             │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_47 (Dense)                │ (32, 512)              │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_15 (Dropout)            │ (32, 512)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_48 (Dense)                │ (32, 128)              │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_49 (Dense)                │ (32, 7)                │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 25,541,927 (97.43 MB)

 Trainable params: 3,739,143 (14.26 MB)

 Non-trainable params: 21,802,784 (83.17 MB)

None


In [63]:
record = model2.fit(
    training_dataset,
    batch_size=32,
    epochs=25,
    verbose=1,
    validation_data=validation_dataset,
    shuffle=True,
)

Epoch 1/25
402/402 ━━━━━━━━━━━━━━━━━━━━ 58s 122ms/step - accuracy: 0.3255 - loss: 1.7679 - val_accuracy: 0.5959 - val_loss: 1.2197
Epoch 2/25
402/402 ━━━━━━━━━━━━━━━━━━━━ 46s 114ms/step - accuracy: 0.5253 - loss: 1.3502 - val_accuracy: 0.6272 - val_loss: 1.1217
Epoch 3/25
402/402 ━━━━━━━━━━━━━━━━━━━━ 46s 115ms/step - accuracy: 0.5640 - loss: 1.2732 - val_accuracy: 0.6150 - val_loss: 1.0979
Epoch 4/25
402/402 ━━━━━━━━━━━━━━━━━━━━ 46s 115ms/step - accuracy: 0.5621 - loss: 1.2406 - val_accuracy: 0.6381 - val_loss: 1.0702
Epoch 5/25
402/402 ━━━━━━━━━━━━━━━━━━━━ 47s 116ms/step - accuracy: 0.5776 - loss: 1.2063 - val_accuracy: 0.6551 - val_loss: 1.0340
Epoch 6/25
402/402 ━━━━━━━━━━━━━━━━━━━━ 46s 115ms/step - accuracy: 0.5815 - loss: 1.1957 - val_accuracy: 0.6510 - val_loss: 1.0116
Epoch 7/25
402/402 ━━━━━━━━━━━━━━━━━━━━ 46s 115ms/step - accuracy: 0.5928 - loss: 1.1723 - val_accuracy: 0.6585 - val_loss: 1.0244
Epoch 8/25
402/402 ━━━━━━━━━━━━━━━━━━━━ 46s 114ms/step - accuracy: 0.6089 - loss: 1

## VGG 16

In [64]:
model3 = create_model(base_model3)

Model: "sequential_24"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential_11 (Sequential)      │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ sequential_12 (Sequential)      │ (32, 224, 224, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ vgg16 (Functional)              │ (32, 7, 7, 512)        │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_10     │ (32, 512)              │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_16 (Dropout)            │ (32, 512)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_50 (Dense)                │ (32, 1024)             │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_51 (Dense)                │ (32, 1024)             │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_52 (Dense)                │ (32, 512)              │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_17 (Dropout)            │ (32, 512)              │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_53 (Dense)                │ (32, 128)              │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_54 (Dense)                │ (32, 7)                │           903 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,880,967 (64.40 MB)

 Trainable params: 9,245,703 (35.27 MB)

 Non-trainable params: 7,635,264 (29.13 MB)

None


In [65]:
record = model3.fit(
    training_dataset,
    batch_size=32,
    epochs=25,
    verbose=1,
    validation_data=validation_dataset,
    shuffle=True
)

Epoch 1/25
402/402 ━━━━━━━━━━━━━━━━━━━━ 86s 206ms/step - accuracy: 0.4414 - loss: 1.5160 - val_accuracy: 0.5490 - val_loss: 1.2337
Epoch 2/25
402/402 ━━━━━━━━━━━━━━━━━━━━ 82s 205ms/step - accuracy: 0.5564 - loss: 1.2675 - val_accuracy: 0.5612 - val_loss: 1.1935
Epoch 3/25
402/402 ━━━━━━━━━━━━━━━━━━━━ 83s 205ms/step - accuracy: 0.5978 - loss: 1.1754 - val_accuracy: 0.6000 - val_loss: 1.1571
Epoch 4/25
402/402 ━━━━━━━━━━━━━━━━━━━━ 82s 204ms/step - accuracy: 0.6143 - loss: 1.1227 - val_accuracy: 0.6286 - val_loss: 1.0734
Epoch 5/25
402/402 ━━━━━━━━━━━━━━━━━━━━ 82s 205ms/step - accuracy: 0.6386 - loss: 1.0575 - val_accuracy: 0.6231 - val_loss: 1.0615
Epoch 6/25
402/402 ━━━━━━━━━━━━━━━━━━━━ 82s 203ms/step - accuracy: 0.6581 - loss: 1.0083 - val_accuracy: 0.6449 - val_loss: 1.0120
Epoch 7/25
402/402 ━━━━━━━━━━━━━━━━━━━━ 83s 205ms/step - accuracy: 0.6711 - loss: 0.9672 - val_accuracy: 0.6422 - val_loss: 0.9939
Epoch 8/25
402/402 ━━━━━━━━━━━━━━━━━━━━ 82s 205ms/step - accuracy: 0.6841 - loss: 0

In [56]:
# loss, accuracy = model.evaluate(test_dataset)
# print(f"Test Loss: {loss}")
# print(f"Test Accuracy: {accuracy}")

In [66]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator




test_datagen = ImageDataGenerator(rescale=None)  


test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(Imagesize, Imagesize),  
    batch_size=32,
    class_mode='categorical',  
    shuffle=False  
)


Found 3195 images belonging to 7 classes.


In [73]:
true_classes = test_generator.classes

In [78]:
predictions = model.predict(test_generator, verbose=1)
predictions2 = model2.predict(test_generator, verbose=1)
predictions3 = model3.predict(test_generator, verbose=1)

100/100 ━━━━━━━━━━━━━━━━━━━━ 31s 310ms/step
100/100 ━━━━━━━━━━━━━━━━━━━━ 30s 306ms/step
100/100 ━━━━━━━━━━━━━━━━━━━━ 30s 300ms/step


### Soft voting

In [75]:
import numpy as np
average_prob = (predictions + predictions2 + predictions3) / 3

y_pred_soft = np.argmax(average_prob, axis=1)

accuracy_soft = np.mean(y_pred_soft == true_classes)
print('Accuracy:', accuracy_soft)

Accuracy: 0.7549295774647887


### Hard Voting

In [84]:
from scipy.stats import mode

preds = np.argmax(predictions, axis=1)
preds2 = np.argmax(predictions2, axis=1)
preds3 = np.argmax(predictions3, axis=1)
max_voting = np.stack([preds, preds2, preds3], axis=1)

max_voting.shape

(3195, 3)

In [85]:
y_pred_hard, _ = mode(max_voting, axis=1)
y_pred_hard = y_pred_hard.ravel()

accuracy = np.mean(y_pred_hard == true_classes)
print('Accuracy:', accuracy)

Accuracy: 0.7339593114241002
